# パターンマッチ

- Elixirの強力な構文
- Elixirにおける`=`演算子はマッチ演算子と呼ばれ、手続き型の言語の代入演算子とは概念が異なる

## マッチ演算子

- 左辺と右辺をマッチさせる
  - 変数を含まない値同士でマッチ可能
- 左辺に変数が含まれる場合、右辺とマッチするよう値に「束縛」される
- 左辺と右辺がマッチしない場合、「MatchError」

In [ ]:
# マッチが成功する例
IO.inspect 0 = 0
IO.inspect "a" = "a"
IO.inspect :a = :a
IO.inspect [0, 1, 2] = [0, 1, 2]
IO.inspect %{a: 0, b: 1} = %{a: 0, b: 1}
IO.inspect %{a: 0} = %{a: 0, b: 1}

以下, MatchErrorとなる例

In [ ]:
0 = 1

In [ ]:
[0, 1] = [0, 1, 2]

In [ ]:
[0, 1, 2] = [0, 1]

In [ ]:
%{a: 0, b: 1} = %{a: 0}

### マッチ演算子と変数束縛

- 変数はマッチ演算子により、右辺にマッチするよう値に束縛される
  - 変数に値を束縛してもマッチできない場合、やはりMatchError
- Elixirでは値は不変(immutable)
  - 変数が束縛された値は、変数を別の値に束縛し直すまで不変

In [ ]:
# 一番最初の時点ではxは定義されていないし、何の値にも束縛されていない
IO.puts x 
# => (CompileError) console:2 "undefined function x/0"

In [ ]:
# 0とマッチさせると、xに0が束縛されていれば両辺がマッチする。よってxに0が束縛される
x = 0

IO.inspect x

In [ ]:
x = 0
0 = x # xは0で束縛されているのでマッチが成功

In [ ]:
x = 0
1 = x # => %MatchError{term: 0} xは既に0に束縛されているので、1とはマッチしない

In [ ]:
# マッチ演算子の両辺が変数の場合
x = 0
y = x
y = 1

array1 = [0, 1, 2]
array2 = array1
array2 = array2 ++ [3]

# 以下では何が出力されるか, 予測して実行してみよう
IO.inspect x == y
IO.inspect x
IO.inspect y

IO.inspect array1 == array2
IO.inspect array1
IO.inspect array2

### Listに対するパターンマッチ

In [ ]:
# 要素が全て等しいのでマッチが成功

l = [0, 1, 2, 3]
[0, 1, 2, 3] = l

In [ ]:
# 変数が含まれる場合、右辺にマッチするよう値が束縛される
[a, b, c, d] = [0, 1, 2, 3]

IO.inspect "#{a}, #{b}, #{c}, #{d}"

In [ ]:
# 右辺が変数を含む場合、変数が束縛されている値にマッチする
x = 10
y = 20
[a, b, c, d] = [0, x, 2, y]

IO.inspect "#{a}, #{b}, #{c}, #{d}"

#### 不要な値を無視する

Elixirにおいて, アンダースコア`_`やアンダースコアで始まる変数は特別な意味を持ち, 「使用しない」変数であることを表す。

Elixirでは未使用の変数に対してコンパイル時にwarningが発せられるが, アンダースコアを用いて使用しないことを明示すればwarningが解消される。

パターンマッチでも変数にマッチはさせるが使用しない場合に, アンダースコアを用いることができる。

In [ ]:
# Listの先頭から3番目の値だけ取り出したいとき

[_, _, x | _tail] = [0, 1, 2, 3]

IO.inspect x

Listに対するパターンマッチでは特有のマッチ記法がある。

`[a | b]` の `a` はリストの先頭、`b` は先頭以外の全てにマッチする。

In [ ]:
[head | tail] = [0, 1, 2, 3, 4]

IO.inspect head
IO.inspect tail

In [ ]:
[head | tail] = [0]

IO.inspect head
IO.inspect tail

In [ ]:
[head | tail] = []
# => %MatchError{term: []}

先頭から$n$個の要素に対してマッチすることも可能。

反対に、最後から$n$個のマッチは不可能。

In [ ]:
[first, second | tail] = [0, 1, 2, 3, 4]

IO.inspect first
IO.inspect second
IO.inspect tail

In [ ]:
[head | last_one_before, last] = [0, 1, 2] # => CompileError

Listの結合演算子`++`を用いてパターンマッチさせることも可能。

ただし、変数を用いる場合は`++`の右辺にしか置けない。

In [ ]:
[0] ++ rest = [0, 1, 2, 3]
IO.inspect rest

In [ ]:
head ++ [1, 2, 3] = [0, 1, 2, 3]
# => CompileError

### Mapに対するパターンマッチ

- 左辺は右辺のサブセットであればいい
- ネストしたマップにもパターンマッチ可能

In [ ]:
%{x: x} = %{x: 0, y: 1, z: 2}
IO.inspect x

In [ ]:
%{:x => x} = %{x: 0, y: 1, z: 2}
IO.inspect x

In [ ]:
%{no_key: value} = %{x: 0, y: 1, z: 2}

In [ ]:
nested_map = %{
  outer_universe: %{
    universe: %{
      hello: "universe!",
      answer_of_everything: 42
    }
  }
}

%{outer_universe: %{universe: %{hello: target}}} = nested_map
IO.inspect target

# マッチ演算子をネストさせることもできる
%{
  outer_universe: %{
    universe: %{
      answer_of_everything: the_answer
    } = universe
  }
} = nested_map

IO.inspect universe
IO.inspect the_answer

### 文字列に対するパターンマッチ

文字列の結合に `<>` という演算子を使用できることを[2_basic_syntax #特徴的な演算子](2_basic_syntax.ipynb#特徴的な演算子)で紹介したが、この演算子はパターンマッチにも使用できる。

ただし、変数を用いる場合は`<>`の右辺にしか置けない。  

In [ ]:
"Hello " <> target = "Hello world!"

IO.inspect target

In [ ]:
greet <> " world" = "Hello world!"
# => ArgumentError

### pin演算子

- 変数は通常、別の値にマッチさせると新しい値に束縛される
- pin演算子を使うと、変数が束縛されている値に対してマッチするか試すことができる

In [ ]:
x = %{a: 0, b: 1}
x = %{a: 0, c: 2} # %{a: 0, b: 1} とマッチしないことを確かめたい
IO.inspect x # xが別のmapに束縛されてしまう

In [ ]:
x = %{a: 0, b: 1}
^x = %{a: 0, c: 2} # xが束縛されているmapには、:cというkeyが含まれないのでMatchError

In [ ]:
# 後で出てくる`case`と組み合わせる例

alexa  = %{name: "Alexa",  listen: "Hello"}
siri   = %{name: "Siri",   listen: "Hey"}
google = %{name: "Google", listen: "OK"}

call_confused_assistant = fn %{name: name} = assistant, call ->
  IO.puts "#{call}, #{name}!"
  
  case assistant do
    # pin演算子を使わないと、変数callが別の値に束縛されてしまう
    %{listen: call} -> IO.puts "#{name} responds to your #{call}"
    _               -> IO.puts "Nothing happens..."
  end
end

call_assistant = fn %{name: name} = assistant, call ->
  IO.puts "#{call}, #{name}!"
  
  case assistant do
    # pin演算子を使うことで変数に束縛された値とマッチするか試すことができる
    %{listen: ^call} -> IO.puts "#{name} responds to your #{call}"
    _                -> IO.puts "Nothing happens..."
  end
end

In [ ]:
call_confused_assistant.(alexa,  "Hey")
call_confused_assistant.(siri,   "Hey")
call_confused_assistant.(google, "Hey")

In [ ]:
call_assistant.(alexa,  "Hey")
call_assistant.(siri,   "Hey")
call_assistant.(google, "Hey")

## 練習問題

パターンマッチで変数に値を束縛し, 論理式が`true`になることを確認してください。

In [ ]:
# 3を変数xに束縛
 = %{a: 1, b: 2, c: 3}

x == 3

In [ ]:
# :aを変数x，2.3を変数yに拘束（1と"a"は何にも拘束しない）
 = [1, :a, "a", 2.3]
 
x == :a and y == 2.3

In [ ]:
# 2を変数x，4を変数y，5～10のリストをzに拘束（1と3は何にも拘束しない）
 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

x == 2 and y == 4 and z == [5, 6, 7, 8, 9, 10]

In [ ]:
# 1を変数x，4を変数yに拘束
 =  %{a: %{b: 1, c: 2, d: [3, 4, 5]}}
 
x == 1 and y == 4

In [ ]:
# 複雑なパターンマッチを試してみよう

request = %{
  header: %{
    "x-custom-header": "a8d3981b2"
  },
  body: %{
    first_name: "Alice",
    last_name:  "Liddell",
    address: [
      "Westminster",
      "London",
      "England",
      "United Kingdom"
    ]
  }
}

# requestのbodyからfirst_nameとlast_nameを同時に取り出してみよう

# requestのbodyのaddressは地区, 州, 構成国, 主権国家の順に並んでいる。州(state)と構成国(country)だけ取り出してみよう
